In [12]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

In [13]:
from fastembed import TextEmbedding

embedding_model = TextEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [14]:
query = input("You: ")
query_vector = list(embedding_model.embed([query]))[0]

In [15]:
results = client.query_points(
    collection_name="source-of-truth",
    query=query_vector,
    limit=5,
    with_payload=True
).points

context = "\n\n".join(
    r.payload["text"] for r in results
)

In [16]:
from openai import OpenAI

llm = OpenAI(
    base_url="http://127.0.0.1:1234/v1",
    api_key="lm-studio"  # dummy key
)

response = llm.chat.completions.create(
    model="meta-llama-3.1-8b-instruct",
    messages=[
        {
            "role": "system",
            "content": "Answer strictly using the provided context."
        },
        {
            "role": "user",
            "content": f"""
Context:
{context}

Question:
{query}
"""
        }
    ],
    temperature=0.4
)

print("Bot:", response.choices[0].message.content)

Bot: According to the provided context, "life imprisonment" is defined as imprisonment for the remainder of a person's natural life.

In different sections:

- Section 264: Imprisonment for life means the remainder of that person’s natural life.
- Section 105: Imprisonment for life is one of the punishments for culpable homicide not amounting to murder, and also for harbouring or concealing a person who has committed an organised crime.
- Section 66: Imprisonment for life is one of the punishments for causing death or resulting in persistent vegetative state of the victim.

In general, "life imprisonment" refers to a prison sentence that lasts until the person's natural death.


In [18]:
for r in results:
    print(r.payload["doc_id"], r.score)

bns2023.pdf 0.7527739
bns2023.pdf 0.74327105
bns2023.pdf 0.7395134
bns2023.pdf 0.7389078
bns2023.pdf 0.7322524
